In [1]:
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline
import numpy as np
import segyio
import tempfile
import os
import matplotlib.pyplot as plt
import winsound
import re
import pandas as pd

In [ ]:
def clip_normalize_cube(cube, percentile=99.5):
    right = np.percentile(cube, percentile)
    left = np.percentile(cube, 100 - percentile)
    bound = np.max([np.abs(left), np.abs(right)])
    np.clip(cube, -bound, bound, cube)
    cube /= bound
    return cube

In [ ]:
def process_volume(volume):
    volume = volume.transpose((0, 2, 1))
    
    #removing outliers
    volume = clip_normalize_cube(volume)
    
    # transforming from [-1.0:1.0] float space to [0:255] uint space
    volume = ((volume + 1) * 255 // 2)
    
    return volume


In [ ]:
def extract_inlines(local_path, save_dir, volume_name, start_idx):
    # Load the volume locally
    volume = segyio.tools.cube(local_path)

    print(f'Volume reading complete, shape: {volume.shape}')

    volume = process_volume(volume)

    # Save inlines as PNG images
    idx = start_idx
    for inline in volume[2:]:  # Limit the loop to the first 100 inlines # volume[:3] in the number you can select the n imag
        plt.imsave(f'{save_dir}/{volume_name}_{idx}.png', inline.astype(int), cmap='gray') # Cmap='',
        idx += 1

    return


In [ ]:
def extract_crosslines(local_path, save_dir, volume_name, start_idx):
    # Load the volume locally
    volume = segyio.tools.cube(local_path)

    print(f'Volume reading complete, shape: {volume.shape}')

    volume = process_volume(volume)

    # Save crosslines as PNG images
    idx = start_idx
    for crossline in volume[:, 2:]:  # Limit the loop to the first 100 crosslines # volume[:3] in the number you can select the n imag
        plt.imsave(f'{save_dir}/{volume_name}_{idx}.png', crossline.astype(int), cmap='gray') # Cmap='',
        idx += 1

    return

In [ ]:
save_path='Parihaka_crosslines'
extract_crosslines(segy_file_path_noise,
                   save_path,                   
                'Pari_clean_cross',
                1)

In [ ]:
save_path='Parihaka_inlines'
extract_inlines(segy_file_path_clean,
                save_path,
                'Pari_clean_inl',
                1)

In [ ]:
from PIL import Image
import os

def crop_and_save(input_dir, output_dir, size=(900, 900)):
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Get a list of all PNG files in the input directory
    png_files = [f for f in os.listdir(input_dir) if f.endswith('.png')]

    for png_file in png_files:
        # Open the image
        img_path = os.path.join(input_dir, png_file)
        img = Image.open(img_path)

        # Get dimensions of the image
        width, height = img.size

        # Calculate the coordinates for cropping
        left = (width - size[0]) / 2
        top = (height - size[1]) / 2
        right = (width + size[0]) / 2
        bottom = (height + size[1]) / 2

        # Crop the image
        cropped_img = img.crop((left, top, right, bottom))

        # Save the cropped image
        output_path = os.path.join(output_dir, png_file)
        cropped_img.save(output_path)

    print("Cropping and saving completed.")

# Usage example
input_directory = "images/clean_enh"
output_directory = "images/noise_enh"
crop_and_save(input_directory, output_directory)

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm 


def add_percentage_gaussian_noise(image, percentage, mean=0, std=85):
    # Calculate the number of pixels to be affected by noise based on the percentage
    num_pixels = int((percentage / 100) * np.prod(image.shape[:2]))

    # Generate random indices for the affected pixels
    indices = np.random.choice(np.prod(image.shape[:2]), num_pixels, replace=False)

    # Convert indices to 2D coordinates
    rows, cols = np.unravel_index(indices, image.shape[:2])

    # Create a mask for the affected pixels
    mask = np.zeros_like(image, dtype=np.uint8)
    mask[rows, cols] = 1

    # Generate random Gaussian noise for the affected pixels
    gaussian_noise = 10*(np.random.normal(mean, std, image.shape))
    noisy_image = image + (gaussian_noise * mask)

    # Clip values to the valid range (0 to 255)
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)

    return noisy_image

def process_images(input_folder, output_folder, noise_percentage):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # List all files in the input folder
    image_files = [f for f in os.listdir(input_folder) if f.endswith('.png')]

    # Process each image file with tqdm for progress bar
    for image_file in tqdm(image_files, desc="Processing images"):
        # Read the image
        image_path = os.path.join(input_folder, image_file)
        original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Apply percentage-based random Gaussian noise
        noisy_image = add_percentage_gaussian_noise(original_image, percentage=noise_percentage)

        # Save the noisy image to the output folder
        output_path = os.path.join(output_folder, f'noisy_{image_file}')
        cv2.imwrite(output_path, noisy_image)



if __name__ == "__main__":
    # Specify the input and output folders
    input_folder ="testes2" #"testes2""images/clean_enh"
    output_folder ="testes2"#"images/noise_enh"
    # Specify the percentage of noise to be added
    noise_percentage = 40

    # Process images with percentage-based random Gaussian noise
    process_images(input_folder, output_folder, noise_percentage)

    print(f'Images processed and saved with {noise_percentage}% random Gaussian noise.')


In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm  # Import tqdm for progress bar

def add_gradient_gaussian_noise(image, percentage, mean=0, std_base=85):
    height, width = image.shape[:2]
    noisy_image = np.copy(image).astype(np.float32)

    # Calculate the number of pixels to be affected by noise based on the percentage
    total_pixels = np.prod(image.shape[:2])
    num_noisy_pixels = int((percentage / 100) * total_pixels)

    # Generate random indices for the affected pixels
    indices = np.random.choice(total_pixels, num_noisy_pixels, replace=False)

    # Apply noise with increasing intensity from top to bottom
    for idx in indices:
        y, x = divmod(idx, width)
        # Calculate standard deviation as a function of y
        std_factor = (y / height) * std_base
        gaussian_noise = np.random.normal(mean, std_factor)

        if len(image.shape) == 3:  # Color image, apply noise to all channels
            noisy_image[y, x, :] += gaussian_noise
        else:  # Grayscale image
            noisy_image[y, x] += gaussian_noise

    # Clip values to the valid range (0 to 255) and convert to uint8
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)

    return noisy_image

def process_images(input_folder, output_folder, noise_percentage):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # List all files in the input folder
    image_files = [f for f in os.listdir(input_folder) if f.endswith('.png')]

    # Process each image file with tqdm for progress bar
    for image_file in tqdm(image_files, desc="Processing images"):
        # Read the image
        image_path = os.path.join(input_folder, image_file)
        original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Apply gradient Gaussian noise
        noisy_image = add_gradient_gaussian_noise(original_image, percentage=noise_percentage)

        # Save the noisy image to the output folder
        output_path = os.path.join(output_folder, f'noisy_{image_file}')
        cv2.imwrite(output_path, noisy_image)

if __name__ == "__main__":
    # Specify the input and output folders
    input_folder = "testes2"  # Example input folder
    output_folder = "testes2"  # Example output folder
    noise_percentage = 50  # Example noise percentage

    # Process images with gradient Gaussian noise
    process_images(input_folder, output_folder, noise_percentage)

    print(f'Images processed and saved with {noise_percentage}% gradient Gaussian noise.')
